# EPA1361 - Model-Based Decision Making
# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version. As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

* Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation. However, we can also look at metrics that give us additional information about the behavior of the model over time. Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design, fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model, and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment
  

* Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50, then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'` of perform_experiments. Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit. In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios, where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`. Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices? Are these sample sizes sufficient for a stable estimation of the indices? You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html) function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function. 

* *hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply slice for 1:50 and 1:250.

* Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point, use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ema_workbench import (Model, MultiprocessingEvaluator, SequentialEvaluator, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging)

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

def pred_prey1(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, 
             final_time=365, reps=1):

    #Initial values
    predators = np.zeros((reps, int(final_time/dt)+1))
    prey = np.zeros((reps, int(final_time/dt)+1))
    sim_time = np.zeros((reps, int(final_time/dt)+1))
    
    for r in range(reps):

        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

    #Calculate the time series
    for t in range(0, sim_time.shape[1]-1):

        dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
        dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

        prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
        predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
        sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

In [2]:
ema_logging.log_to_stderr(ema_logging.INFO)

# Setup the model
pred_prey = Model('PredPreyPy', function=pred_prey1)

pred_prey.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                      RealParameter('predator_efficiency', 0.0001, 0.004),
                      RealParameter('predator_loss_rate', 0.004, 0.08)]

pred_prey.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

In [10]:
from ema_workbench.em_framework.samplers import LHSSampler
from ema_workbench.em_framework.samplers import FullFactorialSampler as FFS

In [13]:
with SequentialEvaluator(pred_prey) as evaluator:
    experiment, outcomes = evaluator.perform_experiments(scenarios = 200, uncertainty_sampling=LHS)

[MainProcess/INFO] performing 200 scenarios * 1 policies * 1 model(s) = 200 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 140 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] experiments finished


In [ ]:
outcomes

In [73]:
df = pd.DataFrame(outcomes['prey'][:,-1])
df['std'] = df.T.std()
df['last'] = df[1460]
df['mean'] = np.mean(df)

In [79]:
Y = df[['std', 'last', 'mean']]




In [81]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X = experiment[['predation_rate', 'predator_efficiency', 'predator_loss_rate', 'prey_birth_rate']]
scaled_data = scale.fit_transform(X)
X = pd.DataFrame(X)
X

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate
0,0.000854,0.002846,0.071857,0.028602
1,0.002213,0.002743,0.035781,0.022772
2,0.002803,0.001202,0.046667,0.027162
3,0.002355,0.001131,0.006751,0.027312
4,0.002724,0.001433,0.007094,0.032842
...,...,...,...,...
195,0.002877,0.000213,0.034494,0.029637
196,0.000998,0.003130,0.072983,0.020003
197,0.001718,0.002779,0.072461,0.020831
198,0.002429,0.000359,0.004009,0.031366


In [82]:
from statsmodels.regression.linear_model import OLS

In [90]:
regression_model = OLS(Y['std'], X).fit()
regression_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    std   R-squared (uncentered):                   0.412
Model:                            OLS   Adj. R-squared (uncentered):              0.400
Method:                 Least Squares   F-statistic:                              34.27
Date:                Mon, 10 May 2021   Prob (F-statistic):                    1.11e-21
Time:                        14:31:17   Log-Likelihood:                         -1008.2
No. Observations:                 200   AIC:                                      2024.
Df Residuals:                     196   BIC:                                      2038.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
predation_rate       -506.1633   3412.562     -0.148      0.882   -7236.218    6223.891
predator_efficiency  -1.43e+04   2304.236     -6.206      0.000   -1.88e+04   -9756.449
predator_loss_rate    367.3323    114.300      3.214      0.002     141.917     592.748
prey_birth_rate      1547.5826    296.772      5.215      0.000     962.306    2132.859
==============================================================================
Omnibus:                      228.057   Durbin-Watson:                   2.273
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8962.445
Skew:                           4.568   Prob(JB):                         0.00
Kurtosis:                      34.497   Cond. No.                         69.2
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [89]:
regression_model = OLS(Y['mean'], X).fit()
regression_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   mean   R-squared (uncentered):                   0.890
Model:                            OLS   Adj. R-squared (uncentered):              0.888
Method:                 Least Squares   F-statistic:                              397.1
Date:                Mon, 10 May 2021   Prob (F-statistic):                    8.73e-93
Time:                        14:31:12   Log-Likelihood:                         -755.11
No. Observations:                 200   AIC:                                      1518.
Df Residuals:                     196   BIC:                                      1531.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
predation_rate       3614.5667    962.684      3.755      0.000    1716.018    5513.116
predator_efficiency  1577.2656    650.025      2.426      0.016     295.324    2859.207
predator_loss_rate    109.5425     32.244      3.397      0.001      45.953     173.132
prey_birth_rate       616.2870     83.719      7.361      0.000     451.180     781.394
==============================================================================
Omnibus:                       10.487   Durbin-Watson:                   0.581
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.400
Skew:                           0.514   Prob(JB):                      0.00552
Kurtosis:                       2.563   Cond. No.                         69.2
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [91]:
regression_model = OLS(Y['last'], X).fit()
regression_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   last   R-squared (uncentered):                   0.379
Model:                            OLS   Adj. R-squared (uncentered):              0.367
Method:                 Least Squares   F-statistic:                              29.96
Date:                Mon, 10 May 2021   Prob (F-statistic):                    1.88e-19
Time:                        14:31:24   Log-Likelihood:                         -1163.9
No. Observations:                 200   AIC:                                      2336.
Df Residuals:                     196   BIC:                                      2349.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
predation_rate       2652.1221   7433.815      0.357      0.722    -1.2e+04    1.73e+04
predator_efficiency -3.393e+04   5019.474     -6.760      0.000   -4.38e+04    -2.4e+04
predator_loss_rate   1017.1626    248.987      4.085      0.000     526.125    1508.200
prey_birth_rate      2630.3328    646.479      4.069      0.000    1355.385    3905.280
==============================================================================
Omnibus:                      317.884   Durbin-Watson:                   2.205
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            52523.437
Skew:                           7.437   Prob(JB):                         0.00
Kurtosis:                      80.984   Cond. No.                         69.2
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [93]:
with SequentialEvaluator(pred_prey) as evaluator:
    experiment_SOBOL, outcomes_SOBOL = evaluator.perform_experiments(scenarios = 50, uncertainty_sampling=SOBOL)

[MainProcess/INFO] performing 500 scenarios * 1 policies * 1 model(s) = 500 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 150 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 250 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] 350 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 450 cases completed
[MainProcess/INFO] 500 cases completed
[MainProcess/INFO] experiments finished
